# 01: Prepare brightway

In this script, we create a new brightway project, import ecoinvent and add all necessary LCIA methods.

In [1]:
%run common_definitions.py

In [5]:
import bw2data as bd
import bw2io as bi
from premise_gwp import add_premise_gwp

import pandas as pd
import os

## Create a new brightway project

In [7]:
premise_version = "2.3.0.dev1"
ei_label = "ecoinvent-{}-cutoff".format(EI_VERSION)
project_name = "premise_runs_{}_{}".format(EI_VERSION, premise_version)

## Import ecoinvent

In [8]:
# import ecoinvent databases
bd.projects.set_current(project_name)
bi.import_ecoinvent_release(
    version=EI_VERSION,
    system_model="cutoff",
    username="pik",
    password="8FragAvBar!"
)

Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds
4718 datasets
0 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/30/2025 11:28:42
  Finished: 07/30/2025 11:28:42
  Total time elapsed: 00:00:00
  CPU %: 86.80
  Memory %: 1.57
Created database: ecoinvent-3.9.1-biosphere


Extracting ecospold2 files:
0% [##############################] 100% | ETA: 00:00:00 | Item ID: fffede58-fed2-5
Total time elapsed: 00:02:49


Title: Extracting ecospold2 files:
  Started: 07/30/2025 11:28:54
  Finished: 07/30/2025 11:31:44
  Total time elapsed: 00:02:49
  CPU %: 82.60
  Memory %: 18.25
Extracted 21238 datasets in 172.34 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_un

Writing activities to SQLite3 database:


21238 datasets
674593 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:31


Title: Writing activities to SQLite3 database:
  Started: 07/30/2025 11:31:50
  Finished: 07/30/2025 11:32:22
  Total time elapsed: 00:00:31
  CPU %: 95.60
  Memory %: 19.03
Created database: ecoinvent-3.9.1-cutoff


## Adding premise_gwp methods

This function adds climate change LCIA methods with biogenic flows.

In [9]:
add_premise_gwp()

RELICS requires the name of your biosphere database.
Please enter the name of your biosphere database as it appears in your project.
Databases dictionary with 2 object(s):
	ecoinvent-3.9.1-biosphere
	ecoinvent-3.9.1-cutoff
Adding ('IPCC 2021', 'climate change', 'GWP 20a, incl. H and bio CO2')
Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.10 seconds
Wrote 1 LCIA methods with 255 characterization factors
Adding ('IPCC 2021', 'climate change', 'GWP 20a, incl. H')
Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Ap

## Create split climate change LCIA methods

Now the newly created climate change LCIA methods are split into the sources `biogenic`, `land use`, and `fossil`.

In [10]:
base_methods = [
    ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2'),
    ('IPCC 2021', 'climate change', 'GWP 20a, incl. H and bio CO2')
]

In [11]:
for method in base_methods:
    M = bd.Method(method)
    unit = ""
    if "unit" in M.metadata:
        unit = M.metadata["unit"]

    CFs_bio = []
    CFs_landuse = []
    CFs_fossil = []
    for flow in bd.Method(method).load():
        act = bd.get_activity(flow[0])
        if "non-fossil" in act["name"] or "in air" in act["name"]:
            CFs_bio.append(list(flow))
        elif "soil or biomass" in act["name"]:
            CFs_landuse.append(list(flow))
        else:
            CFs_fossil.append(list(flow))

    new_key = (method[0], method[1]+": biogenic", method[2].split(",")[0])    
    new_method = bd.Method(new_key)
    new_method.validate(CFs_bio)
    new_method.register(unit=unit)
    new_method.write(CFs_bio)

    new_key = (method[0], method[1]+": land use", method[2].split(",")[0])    
    new_method = bd.Method(new_key)
    new_method.validate(CFs_landuse)
    new_method.register(unit=unit)
    new_method.write(CFs_landuse)

    new_key = (method[0], method[1]+": fossil", method[2].split(",")[0])    
    new_method = bd.Method(new_key)
    new_method.validate(CFs_fossil)
    new_method.register(unit=unit)
    new_method.write(CFs_fossil)

## Add the Ecocosts Materials method

In [12]:
table = pd.read_csv("../data/cfs_ecocosts_metals.csv")
table.dropna(subset="characterization factor", inplace=True)

In [13]:
db = bd.Database("ecoinvent-3.9.1-biosphere")
resources = [flow for flow in db if flow["categories"] == ("natural resource", "in ground")]

new_key = ("Eco-costs", "Material scarcity", "VAR(95)")
CFs = []
for idx, row in table.iterrows():
    matches = [flow for flow in resources if flow["name"] == row["name"] and str(flow["categories"]) == row["categories"]]
    if len(matches) == 1:
        CFs.append([matches[0].key, row["characterization factor"]])
    else:
        print("Error: More than one match.")

new_method = bd.Method(new_key)
new_method.validate(CFs)
new_method.register(unit="EUR2023")
new_method.write(CFs)

## Add the old ReCiPe methods

Load CFs.

In [14]:
df_recipe = pd.read_csv("../data/cfs_old_recipe_methods_mapped.csv")

Find matching biosphere flows

In [15]:
db = bd.Database("ecoinvent-3.9.1-biosphere")

flowlist = []
categorieslist = []
keylist = []

needed_flows = list(df_recipe.groupby(["new flow name", "flow categories"]).size().index)

for j, f in enumerate(needed_flows):
    matching_names = [act for act in db if act["name"] == f[0]]
    matching_cats = [act for act in matching_names if str(act["categories"]) == f[1]]
    if len(matching_cats) == 1:
        flowlist.append(f[0])
        categorieslist.append(f[1])
        keylist.append(matching_cats[0].key)
    print("{} of {} flows checked.".format(j+1, len(needed_flows)))

flow_mapping = pd.DataFrame(
    {
        "new flow name": flowlist,
        "flow categories": categorieslist,
        "mapped key": keylist
    }
)

1 of 2477 flows checked.
2 of 2477 flows checked.
3 of 2477 flows checked.
4 of 2477 flows checked.
5 of 2477 flows checked.
6 of 2477 flows checked.
7 of 2477 flows checked.
8 of 2477 flows checked.
9 of 2477 flows checked.
10 of 2477 flows checked.
11 of 2477 flows checked.
12 of 2477 flows checked.
13 of 2477 flows checked.
14 of 2477 flows checked.
15 of 2477 flows checked.
16 of 2477 flows checked.
17 of 2477 flows checked.
18 of 2477 flows checked.
19 of 2477 flows checked.
20 of 2477 flows checked.
21 of 2477 flows checked.
22 of 2477 flows checked.
23 of 2477 flows checked.
24 of 2477 flows checked.
25 of 2477 flows checked.
26 of 2477 flows checked.
27 of 2477 flows checked.
28 of 2477 flows checked.
29 of 2477 flows checked.
30 of 2477 flows checked.
31 of 2477 flows checked.
32 of 2477 flows checked.
33 of 2477 flows checked.
34 of 2477 flows checked.
35 of 2477 flows checked.
36 of 2477 flows checked.
37 of 2477 flows checked.
38 of 2477 flows checked.
39 of 2477 flows chec

In [16]:
df_recipe_mapped = pd.concat(
    (
        df_recipe.set_index(["new flow name", "flow categories"]),
        flow_mapping.set_index(["new flow name", "flow categories"])
    ),
    axis=1
).reset_index()

df_recipe_mapped[["method", "unit", "new flow name", "flow categories", "mapped key", "CF"]].to_csv("../data/cfs_old_recipe_methods_mapped_w_keys.csv", index=False)

Add the methods to brightway:

In [17]:
recipe_cfs = pd.read_csv("../data/cfs_old_recipe_methods_mapped_w_keys.csv").dropna(subset="mapped key")
methods2add = list(recipe_cfs["method"].unique())

for m in methods2add:
    sel = recipe_cfs[recipe_cfs["method"] == m].copy()
    new_key = tuple(list(m.split(", ")))
    unit = sel["unit"].iloc[0]
    keys = list(sel["mapped key"])
    cfs = list(sel["CF"])
    new_cfs = [[eval(k), cf] for k, cf in zip(keys, cfs)]
    new_method = bd.Method(new_key)
    new_method.validate(new_cfs)
    new_method.register(unit=unit)
    new_method.write(new_cfs)

    print("Added method {}".format(str(new_method)))

Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: freshwater ecotoxicity w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: freshwater eutrophication w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: urban land occupation w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: climate change: ecosystems w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: terrestrial ecotoxicity w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: agricultural land occupation w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: marine ecotoxicity w/o LT
Added method Brightway2 Method: ReCiPe Endpoint (E,A) w/o LT: ecosystem quality w/o LT: natural land transformation w/o LT
Added method Brightway2 Method: ReCiPe End